In [ ]:
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt

API_KEY = "eesWo2cNaJU0GbkFGXWhwDkxlVPSes4Y1rKA0sA4"




In [ ]:
'''
API endpoint

https://api.eia.gov/v2/electricity/rto/region-data/data/

Key request components:

    frequency — hourly/daily/monthly

    data[0] — which metric you want, e.g., "value"

    facets[...] — filters (e.g., region, type, etc.)

    sort[...][column] — sort by period

    length — max rowsb returned (we’ll limit to 5000)

'''

In [34]:


BASE_URL = "https://api.eia.gov/v2/electricity/rto/daily-fuel-type-data/data/"

params = {
    "api_key": API_KEY,
    "frequency": "daily",
    "data[0]": "value",
    "facets[fueltype][]": ["BAT","COL","GEO","NG","NUC","OIL","SNB","SUN","WAT","WND"],
    "facets[respondent][]": ["CISO","ERCO","NYIS"],
    "start": "2025-01-01",
    "end": "2025-02-28",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "offset": 0,
    "length": 5000
}

response = requests.get(BASE_URL, params=params)
json_data = response.json()

df = pd.DataFrame(json_data["response"]["data"])
df.head()


,period,respondent,respondent-name,fueltype,type-name,timezone,timezone-description,value,value-units
0,2025-02-28,CISO,California Independent System Operator,COL,Coal,Arizona,Arizona,-35,megawatthours
1,2025-02-28,CISO,California Independent System Operator,COL,Coal,Central,Central,-35,megawatthours
2,2025-02-28,CISO,California Independent System Operator,COL,Coal,Eastern,Eastern,-35,megawatthours
3,2025-02-28,CISO,California Independent System Operator,COL,Coal,Mountain,Mountain,-35,megawatthours
4,2025-02-28,CISO,California Independent System Operator,COL,Coal,Pacific,Pacific,-36,megawatthours


In [35]:
# Ensure value is numeric
df["value"] = pd.to_numeric(df["value"], errors="coerce")

# Convert "period" to datetime
df["period"] = pd.to_datetime(df["period"])

# Extract Year-Month
df["month"] = df["period"].dt.to_period("M").astype(str)


In [ ]:
monthly_totals = (
    df.groupby(["month", "fueltype"])["value"]
      .sum()
      .reset_index()
)

monthly_totals["monthly_total"] = (
    monthly_totals.groupby("month")["value"]
    .transform("sum")
)

monthly_totals["share_pct"] = (
    monthly_totals["value"] / monthly_totals["monthly_total"] * 100
)

monthly_share = (
    monthly_totals
    .sort_values(["month", "share_pct"], ascending=[True, False])
)

monthly_share


,month,fueltype,value,monthly_total,share_pct
2,2025-01,NG,100180009,215868150,46.407962
7,2025-01,WND,36448216,215868150,16.884481
3,2025-01,NUC,25083515,215868150,11.619831
1,2025-01,COL,21156889,215868150,9.800839
5,2025-01,SUN,20899962,215868150,9.681818
6,2025-01,WAT,10597612,215868150,4.909299
0,2025-01,BAT,1308793,215868150,0.606293
4,2025-01,OIL,193154,215868150,0.089478
10,2025-02,NG,125489661,291923063,42.987238
15,2025-02,WND,55132008,291923063,18.885801
